# Подготовка фичей (для модели на различиях атрибутов)

<br><br><br>
# Названия файлов и путей

In [1]:
# 4COLAB: подключаем гугл-диск
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
# путь до папки с файлами
files_path = '/content/drive/MyDrive/Colab Notebooks/Ozon/datasets/'

# ----- INPUT FILES --------------------------------------------------
# названия исходных файлов обучающей выборки
trn_data_file = 'train_data.parquet'
trn_pairs_file = 'train_pairs.parquet'

# названия исходных файлов тестовой выборки
tst_data_file = 'test_data.parquet'
tst_pairs_file = 'test_pairs_wo_target.parquet'

# ----- OUTPUT FILES -------------------------------------------------
# названия выходных файлов с фичами обучающей и тестовой выборок
trn_features_file = 'train_attr_features_01.parquet'
tst_features_file = 'test_attr_features_01.parquet'

<br><br><br>
# Подготовка к работе

In [3]:
# импорт необходимых модулей
import pandas as pd
import numpy as np
import json
import time

import warnings

In [4]:
# предупреждения OFF
warnings.filterwarnings('ignore')

In [5]:
# настройки вывода датафреймов
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 50)

In [6]:
# функция возвращает разницу во времени в виде строки '(**m **s)' или '(<1s)'
def get_time(t1, t2):
    dt = t2 - t1
    if int(dt):
        return '(' + (f'{dt//60:.0f}m ' if dt//60 else '') + f'{dt%60:.0f}s)'
    return '(<1s)'

In [7]:
# функция запуска таймера
def timer_start(msg=''):
    timer = time.time()
    print(msg, end='')
    return timer

In [8]:
# функция остановка таймера
def timer_stop(start_time, msg='Готово!'):
    print(f'{msg} {get_time(start_time, time.time())}')

In [9]:
# функция выводит датафреймы на экран
def show_dfs(dfs, txts, head=5):
    for df, txt in zip(dfs, txts):
        print(f'{txt}:', df.shape)
        display(df.head(head))
        print()

In [10]:
# старт таймера всего скрипта
start_nb = timer_start()

<br><br><br>
# Загрузка и подготовка датафреймов

In [11]:
%%time
# загрузка файлов обучающей выборки
trn_data = pd.read_parquet(files_path + trn_data_file)
trn_df = pd.read_parquet(files_path + trn_pairs_file)

# Удаляем лишние столбцы и переименовываем столбец с атрибутами
trn_data = trn_data[['variantid', 'characteristic_attributes_mapping', 'categories']]
trn_data.columns = ['id', 'attr', 'cat3_grouped']

CPU times: user 15.6 s, sys: 11.9 s, total: 27.5 s
Wall time: 36 s


In [12]:
%%time
# загрузка файлов тестовой выборки
tst_data = pd.read_parquet(files_path + tst_data_file)
tst_df = pd.read_parquet(files_path + tst_pairs_file)

# Удаляем лишние столбцы и переименовываем столбец с атрибутами
tst_data = tst_data[['variantid', 'characteristic_attributes_mapping', 'categories']]
tst_data.columns = ['id', 'attr', 'cat3_grouped']

CPU times: user 984 ms, sys: 502 ms, total: 1.49 s
Wall time: 2.59 s


In [13]:
# out (train)
show_dfs([trn_data, trn_df], ['train_data', 'train_pairs'])

train_data: (457063, 3)


,id,attr,cat3_grouped
0,51195767,"{""Номинальный ток, А"":[""10""],""Цвет товара"":[""оранжевый""],""Входное напряжение, В"":[""220""],""Макс. ток, А"":[""10""],""Бренд"":[""Партнер-Электро""],""Вес товара, г"":[""1760""],""Вид розеток"":[""Евровилка""],""Эле...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые фильтры, разветвители и удлинители"", ""4"": ""Сетевой фильтр, удлинитель, разветвитель""}"
1,53565809,"{""Конструктивные особенности"":[""Магнитная конструкция коннектора""],""Цвет товара"":[""красный""],""Тип коннектора 1"":[""Штекер""],""Макс. ток, А"":[""2""],""Бренд"":[""Baseus""],""Коннектор 1"":[""USB 2.0 Type-A""],...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Кабели и переходники"", ""4"": ""Кабель""}"
2,56763357,"{""Тип аксессуара"":[""Набор микропрепаратов""],""Бренд"":[""Konus""],""Гарантийный срок"":[""2 года""],""Комплектация"":[""Набор микропрепаратов Konus 25: «Беспозвоночные и насекомые»\nОдноклеточный организм (а...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Оптические приборы"", ""4"": ""Микроскоп""}"
3,56961772,"{""Тип карты памяти"":[""microSD""],""Число SIM-карт"":[""2""],""Функциональные особенности телефона"":[""Диктофон""],""Форм-фактор SIM"":[""Mini-SIM""],""Цвет товара"":[""черный""],""Встроенная камера"":[""Нет""],""Диапа...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Смартфоны, планшеты, мобильные телефоны"", ""4"": ""Мобильный телефон""}"
4,61054740,"{""Материал"":[""Металл""],""Количество секций, шт"":[""4""],""Цвет товара"":[""черный""],""Бренд"":[""Apiko""],""Гарантийный срок"":[""1 год""],""Особенности"":[""Возможность съемки панорам"",""Съемная площадка"",""Уровень...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Штативы и головки"", ""4"": ""Штатив""}"



train_pairs: (306540, 3)


,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289
2,1.0,53602615,587809782
3,1.0,53888651,89598677
4,0.0,56930698,551526166


In [14]:
# out (test)
show_dfs([tst_data, tst_df], ['test_data', 'test_pairs'])

test_data: (35730, 3)


,id,attr,cat3_grouped
0,51201254,"{""Страна-изготовитель"":[""Китай""],""Бренд"":[""TDM Electric""],""Входное напряжение, В"":[""250""],""Количество розеток, шт"":[""4""],""Тип"":[""Удлинитель бытовой""],""Номинальный ток, А"":[""10""],""Стандарт защиты"":...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые фильтры, разветвители и удлинители"", ""4"": ""Сетевой фильтр, удлинитель, разветвитель""}"
1,77151532,"{""Страна-изготовитель"":[""Китай""],""Комплектация"":["" с черной рамкой""],""Бренд"":[""CHIP""],""Вес товара, г"":[""120""],""Гарантийный срок"":[""14 дней""],""Тип"":[""Клавиатура для ноутбука""],""Цвет товара"":[""черны...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Запчасти для ноутбуков"", ""4"": ""Клавиатура для ноутбука""}"
2,89664856,"{""Видеокарта"":[""NVIDIA GeForce RTX 2070 (8 Гб)""],""Процессор"":[""Intel Core i7-10750H (2.6 ГГц)""],""Тип аккумулятора"":[""Li-ion""],""Количество SSD"":[""2""],""Конфигурация звука"":[""Stereo""],""Особенности ус...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Компьютер"", ""4"": ""Ноутбук""}"
3,90701982,"{""Основной материал корпуса"":[""Металл""],""Максимальная мощность, Вт"":[""10""],""Декоративное покрытие корпуса"":[""Пластик""],""FM-радио"":[""Да""],""Страна-изготовитель"":[""Китай""],""Конструктивные особенности...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Акустика и колонки"", ""4"": ""Портативная акустическая система""}"
4,92484118,"{""Рекомендовано для"":[""Meizu""],""Бренд"":[""Meizu""],""Список совместимых устройств"":[""M6s""],""Тип"":[""Аккумулятор для телефона""],""Вес с упаковкой, г"":[""250""]}","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарейки и аккумуляторы"", ""4"": ""Аккумулятор для телефона""}"



test_pairs: (18084, 2)


,variantid1,variantid2
0,52076340,290590137
1,64525522,204128919
2,77243372,479860557
3,86065820,540678372
4,91566575,258840506


<br><br><br>
# Извлечение категорий 3-го уровня, объединение мелких в rest

In [15]:
%%time
# извлечение категории 3 уровня из словаря категорий
trn_data['cat3_grouped'] = trn_data['cat3_grouped'].apply(lambda x: json.loads(x)['3'])
tst_data['cat3_grouped'] = tst_data['cat3_grouped'].apply(lambda x: json.loads(x)['3'])

CPU times: user 1.56 s, sys: 0 ns, total: 1.56 s
Wall time: 1.55 s


In [16]:
# out
show_dfs([trn_data, tst_data], ['train_data', 'test_data'])

train_data: (457063, 3)


,id,attr,cat3_grouped
0,51195767,"{""Номинальный ток, А"":[""10""],""Цвет товара"":[""оранжевый""],""Входное напряжение, В"":[""220""],""Макс. ток, А"":[""10""],""Бренд"":[""Партнер-Электро""],""Вес товара, г"":[""1760""],""Вид розеток"":[""Евровилка""],""Эле...","Сетевые фильтры, разветвители и удлинители"
1,53565809,"{""Конструктивные особенности"":[""Магнитная конструкция коннектора""],""Цвет товара"":[""красный""],""Тип коннектора 1"":[""Штекер""],""Макс. ток, А"":[""2""],""Бренд"":[""Baseus""],""Коннектор 1"":[""USB 2.0 Type-A""],...",Кабели и переходники
2,56763357,"{""Тип аксессуара"":[""Набор микропрепаратов""],""Бренд"":[""Konus""],""Гарантийный срок"":[""2 года""],""Комплектация"":[""Набор микропрепаратов Konus 25: «Беспозвоночные и насекомые»\nОдноклеточный организм (а...",Оптические приборы
3,56961772,"{""Тип карты памяти"":[""microSD""],""Число SIM-карт"":[""2""],""Функциональные особенности телефона"":[""Диктофон""],""Форм-фактор SIM"":[""Mini-SIM""],""Цвет товара"":[""черный""],""Встроенная камера"":[""Нет""],""Диапа...","Смартфоны, планшеты, мобильные телефоны"
4,61054740,"{""Материал"":[""Металл""],""Количество секций, шт"":[""4""],""Цвет товара"":[""черный""],""Бренд"":[""Apiko""],""Гарантийный срок"":[""1 год""],""Особенности"":[""Возможность съемки панорам"",""Съемная площадка"",""Уровень...",Штативы и головки



test_data: (35730, 3)


,id,attr,cat3_grouped
0,51201254,"{""Страна-изготовитель"":[""Китай""],""Бренд"":[""TDM Electric""],""Входное напряжение, В"":[""250""],""Количество розеток, шт"":[""4""],""Тип"":[""Удлинитель бытовой""],""Номинальный ток, А"":[""10""],""Стандарт защиты"":...","Сетевые фильтры, разветвители и удлинители"
1,77151532,"{""Страна-изготовитель"":[""Китай""],""Комплектация"":["" с черной рамкой""],""Бренд"":[""CHIP""],""Вес товара, г"":[""120""],""Гарантийный срок"":[""14 дней""],""Тип"":[""Клавиатура для ноутбука""],""Цвет товара"":[""черны...",Запчасти для ноутбуков
2,89664856,"{""Видеокарта"":[""NVIDIA GeForce RTX 2070 (8 Гб)""],""Процессор"":[""Intel Core i7-10750H (2.6 ГГц)""],""Тип аккумулятора"":[""Li-ion""],""Количество SSD"":[""2""],""Конфигурация звука"":[""Stereo""],""Особенности ус...",Компьютер
3,90701982,"{""Основной материал корпуса"":[""Металл""],""Максимальная мощность, Вт"":[""10""],""Декоративное покрытие корпуса"":[""Пластик""],""FM-радио"":[""Да""],""Страна-изготовитель"":[""Китай""],""Конструктивные особенности...",Акустика и колонки
4,92484118,"{""Рекомендовано для"":[""Meizu""],""Бренд"":[""Meizu""],""Список совместимых устройств"":[""M6s""],""Тип"":[""Аккумулятор для телефона""],""Вес с упаковкой, г"":[""250""]}",Батарейки и аккумуляторы


In [17]:
%%time
# train: порог, меньше которого категории объединяются в rest
trn_cat_threshold = 1000

# train: список маленьких категорий с кол-вом товаров < trn_cat_threshold
trn_lil_cats = trn_data['cat3_grouped'].value_counts()
trn_lil_cats = trn_lil_cats[trn_lil_cats < trn_cat_threshold]

# train: сливаем маленькие категории в категорию rest
trn_data.loc[trn_data['cat3_grouped'].isin(trn_lil_cats.index), 'cat3_grouped'] = 'rest'

CPU times: user 119 ms, sys: 0 ns, total: 119 ms
Wall time: 121 ms


In [18]:
%%time
# test: порог, меньше которого категории объединяются в rest
tst_cat_threshold = 50

# test: список маленьких категорий с кол-вом товаров < tst_cat_threshold
tst_lil_cats = tst_data['cat3_grouped'].value_counts()
tst_lil_cats = tst_lil_cats[tst_lil_cats < tst_cat_threshold]

# test: сливаем маленькие категории в категорию rest
tst_data.loc[tst_data['cat3_grouped'].isin(tst_lil_cats.index), 'cat3_grouped'] = 'rest'

CPU times: user 8.86 ms, sys: 0 ns, total: 8.86 ms
Wall time: 9.29 ms


In [19]:
# out
print('train data:')
print(f'Категорий 3-го уровня с количеством товаров < {trn_cat_threshold}:', len(trn_lil_cats.index))
print('Например:', "'" + "', '".join(trn_lil_cats.index[:5]) + "', ...")
print('\ntest data:')
print(f'Категорий 3-го уровня с количеством товаров < {tst_cat_threshold}:', len(tst_lil_cats.index))
print('Например:', "'" + "', '".join(tst_lil_cats.index[:5]) + "', ...")

train data:
Категорий 3-го уровня с количеством товаров < 1000: 82
Например: 'Расходные материалы', 'Штативы и головки', 'Усилители, ресиверы и ЦАПы', 'Запчасти для аудио/видеотехники', 'Проектор', ...

test data:
Категорий 3-го уровня с количеством товаров < 50: 22
Например: 'Охранная система', 'ТВ-приставки и медиаплееры', 'Электронное оборудование для торговли', 'Проектор', 'Запчасти и аксессуары для проекторов', ...


In [20]:
# out
show_dfs([trn_data, tst_data], ['train_data', 'test_data'])

train_data: (457063, 3)


,id,attr,cat3_grouped
0,51195767,"{""Номинальный ток, А"":[""10""],""Цвет товара"":[""оранжевый""],""Входное напряжение, В"":[""220""],""Макс. ток, А"":[""10""],""Бренд"":[""Партнер-Электро""],""Вес товара, г"":[""1760""],""Вид розеток"":[""Евровилка""],""Эле...","Сетевые фильтры, разветвители и удлинители"
1,53565809,"{""Конструктивные особенности"":[""Магнитная конструкция коннектора""],""Цвет товара"":[""красный""],""Тип коннектора 1"":[""Штекер""],""Макс. ток, А"":[""2""],""Бренд"":[""Baseus""],""Коннектор 1"":[""USB 2.0 Type-A""],...",Кабели и переходники
2,56763357,"{""Тип аксессуара"":[""Набор микропрепаратов""],""Бренд"":[""Konus""],""Гарантийный срок"":[""2 года""],""Комплектация"":[""Набор микропрепаратов Konus 25: «Беспозвоночные и насекомые»\nОдноклеточный организм (а...",Оптические приборы
3,56961772,"{""Тип карты памяти"":[""microSD""],""Число SIM-карт"":[""2""],""Функциональные особенности телефона"":[""Диктофон""],""Форм-фактор SIM"":[""Mini-SIM""],""Цвет товара"":[""черный""],""Встроенная камера"":[""Нет""],""Диапа...","Смартфоны, планшеты, мобильные телефоны"
4,61054740,"{""Материал"":[""Металл""],""Количество секций, шт"":[""4""],""Цвет товара"":[""черный""],""Бренд"":[""Apiko""],""Гарантийный срок"":[""1 год""],""Особенности"":[""Возможность съемки панорам"",""Съемная площадка"",""Уровень...",rest



test_data: (35730, 3)


,id,attr,cat3_grouped
0,51201254,"{""Страна-изготовитель"":[""Китай""],""Бренд"":[""TDM Electric""],""Входное напряжение, В"":[""250""],""Количество розеток, шт"":[""4""],""Тип"":[""Удлинитель бытовой""],""Номинальный ток, А"":[""10""],""Стандарт защиты"":...","Сетевые фильтры, разветвители и удлинители"
1,77151532,"{""Страна-изготовитель"":[""Китай""],""Комплектация"":["" с черной рамкой""],""Бренд"":[""CHIP""],""Вес товара, г"":[""120""],""Гарантийный срок"":[""14 дней""],""Тип"":[""Клавиатура для ноутбука""],""Цвет товара"":[""черны...",Запчасти для ноутбуков
2,89664856,"{""Видеокарта"":[""NVIDIA GeForce RTX 2070 (8 Гб)""],""Процессор"":[""Intel Core i7-10750H (2.6 ГГц)""],""Тип аккумулятора"":[""Li-ion""],""Количество SSD"":[""2""],""Конфигурация звука"":[""Stereo""],""Особенности ус...",Компьютер
3,90701982,"{""Основной материал корпуса"":[""Металл""],""Максимальная мощность, Вт"":[""10""],""Декоративное покрытие корпуса"":[""Пластик""],""FM-радио"":[""Да""],""Страна-изготовитель"":[""Китай""],""Конструктивные особенности...",Акустика и колонки
4,92484118,"{""Рекомендовано для"":[""Meizu""],""Бренд"":[""Meizu""],""Список совместимых устройств"":[""M6s""],""Тип"":[""Аккумулятор для телефона""],""Вес с упаковкой, г"":[""250""]}",Батарейки и аккумуляторы


<br><br><br>
# Извлечение вектора атрибутов

In [21]:
%%time
# извлечение вектора атрибутов (только из обучающей выборки)
attr_series = trn_data['attr'].apply(lambda x: set(json.loads(x).keys()) if x != None else set())
attr_list = sorted(list(set().union(*attr_series)))

CPU times: user 11.8 s, sys: 196 ms, total: 12 s
Wall time: 12.2 s


In [22]:
# out
print('Всего извлечено уникальных атрибутов:', len(attr_list))  # 1447
print('\nПервые 15:')
attr_list[:15]

Всего извлечено уникальных атрибутов: 1447

Первые 15:


['Active to Precharge Delay (tRAS)',
 'Bluetooth',
 'CAS Latency (CL)',
 'Cтандарты HDR',
 'FM-радио',
 'MTBF, кликов',
 'OEM-номер',
 'RAID',
 'RAS to CAS Delay (tRCD)',
 'Row Precharge Delay (tRP)',
 'Smart TV',
 'True Wireless',
 'Web-камера',
 'Автомат',
 'Автоматика телескопа']

<br><br><br>
# Мержинг. Добавление в парные датафреймы столбцов атрибутов товаров 1 и 2

In [23]:
%%time
# мержинг обучающей выборки
trn_df = trn_df.merge(trn_data, left_on='variantid1', right_on='id', how='left')
trn_df = trn_df.merge(trn_data, left_on='variantid2', right_on='id', how='left', suffixes=('1', '2'))

# удаление ненужных столбцов, переименование столбца с категориями
trn_df.drop(['id1', 'id2', 'cat3_grouped2'], axis=1, inplace=True)
trn_df.rename({'cat3_grouped1': 'cat3_grouped'}, axis=1, inplace=True)

CPU times: user 687 ms, sys: 38.4 ms, total: 725 ms
Wall time: 731 ms


In [24]:
%%time
# мержинг тестовой выборки
tst_df = tst_df.merge(tst_data, left_on='variantid1', right_on='id', how='left')
tst_df = tst_df.merge(tst_data, left_on='variantid2', right_on='id', how='left', suffixes=('1', '2'))

# удаление ненужных столбцов, переименование столбца с категориями
tst_df.drop(['id1', 'id2', 'cat3_grouped2'], axis=1, inplace=True)
tst_df.rename({'cat3_grouped1': 'cat3_grouped'}, axis=1, inplace=True)

CPU times: user 28.5 ms, sys: 0 ns, total: 28.5 ms
Wall time: 28.1 ms


In [25]:
# out
show_dfs([trn_df, tst_df], ['train_df', 'test_df'], 2)

train_df: (306540, 6)


,target,variantid1,variantid2,attr1,cat3_grouped,attr2
0,0.0,51197862,51198054,"{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопасность"":[""Заземление""],""Количество розеток, шт"":[""5""],...","Сетевые фильтры, разветвители и удлинители","{""Электробезопасность"":[""Заземление""],""Длина кабеля питания, м"":[""1.5""],""Входное напряжение, В"":[""250""],""Вид кабеля"":[""ПВС""],""Тип"":[""Удлинитель бытовой""],""Сечение жилы, кв.мм"":[""1""],""Количество ро..."
1,1.0,53062686,536165289,"{""Количество в упаковке, шт"":[""1""],""Бренд"":[""Комус""],""Назначение"":[""для лазерного принтера""],""Совместимые модели принтеров"":[""Canon i-SENSYS LBP7018C""],""Бренд печатающего устройства"":[""Canon""],""Цв...",Расходник для печати,"{""Бренд"":[""Комус""],""Тип"":[""Картридж""]}"



test_df: (18084, 5)


,variantid1,variantid2,attr1,cat3_grouped,attr2
0,52076340,290590137,"{""Напряжение, В"":[""1.5""],""Бренд"":[""Perfeo""],""Тип"":[""Батарейка""],""Форм-фактор батареи"":[""AAA""],""Химический тип"":[""Щелочной""],""Страна-изготовитель"":[""Китай""],""Гарантийный срок"":[""31 день (с учетом с...",Батарейки и аккумуляторы,"{""Форм-фактор батареи"":[""AAA""],""Химический тип"":[""Щелочной""],""Назначение"":[""Универсальный""],""Комплектация"":[""Батарейка, фирменная упаковка""],""Количество в упаковке, шт"":[""4""],""Страна-изготовитель""..."
1,64525522,204128919,"{""Операционная система"":[""Android""],""Защищенность"":[""Влагозащита"",""Ударопрочный корпус""],""Навигация"":[""A-GPS"",""GPS"",""ГЛОНАСС""],""Модель процессора"":[""Helio P23""],""Видеопроцессор"":[""Mali-G71 MP2""],""...","Смартфоны, планшеты, мобильные телефоны","{""Встроенная память"":[""32 ГБ""],""Видеопроцессор"":[""Mali-400 MP1""],""Степень защиты"":[""IP68"",""IP69K""],""Количество основных камер"":[""2""],""Бренд"":[""Ulefone""],""Вес товара, г"":[""235""],""Тип карты памяти"":..."


<br><br><br>
# Добавление столбцов всех атрибутов и заполнение фичами

In [26]:
%%time
# Создаем в датафрейме столбцы по названиям атрибутов, заполняем нулями
trn_df[attr_list] = 0.
tst_df[attr_list] = 0.

CPU times: user 1.62 s, sys: 2.59 s, total: 4.2 s
Wall time: 4.2 s


In [27]:
# out
print('Обучающая выборка:')
print(trn_df.info())
print('\nТестовая выборка:')
print(tst_df.info())

Обучающая выборка:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 306540 entries, 0 to 306539
Columns: 1453 entries, target to Яркость, кд/м2
dtypes: float64(1448), int64(2), object(3)
memory usage: 3.3+ GB
None

Тестовая выборка:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18084 entries, 0 to 18083
Columns: 1452 entries, variantid1 to Яркость, кд/м2
dtypes: float64(1447), int64(2), object(3)
memory usage: 200.5+ MB
None


In [28]:
# out
print('Размерность датафреймов обучающей итестовой выборок: ', trn_df.shape, tst_df.shape)

Размерность датафреймов обучающей итестовой выборок:  (306540, 1453) (18084, 1452)


In [29]:
# функция заполнения столбцов атрибутов значениями совпадений (или нет) атрибутов
def fill_columns(row):
    attr1 = json.loads(row['attr1']) if row['attr1'] != None else dict()
    attr2 = json.loads(row['attr2']) if row['attr2'] != None else dict()
    # цикл по совпадающим в 2 словарях ключам
    for key in attr1.keys() & attr2.keys():
        # if ключ есть в столбцах
        if key in row.index:

            # # если совпадают, в т.ч. списки независимо от порядка (same)
            # if set(attr1[key]) == set(attr2[key]):
            #     row[key] = 2
            #     continue

            # если нет пересечений в списках, независимо от дилины (diff)
            if not set(attr1[key]) & set(attr2[key]):
                row[key] = 1
                continue

            # # если всё же есть пересечения, но в целом списки по составу не совпадают
            # row[key] = 0.5

    return row

In [30]:
# расчёт фичей в обучающей выборке
start0 = timer_start('Расчёт фичей в обучающей выборке\n\n')

# расчет количества батчей
nrows = trn_df.shape[0]
batch_size = 50_000
nbatches = nrows // batch_size + (nrows % batch_size != 0) * 1
print(f'Строк в выборке: {nrows}')
print(f'Размер батча:    {batch_size}')
print(f'Всего батчей:    {nbatches}\n')

# заполнение фичей
print('Обработка батча:')
for i in range(nbatches):
    start = timer_start(f'{i+1:03}:  ')
    idx1 = i * batch_size
    idx2 = idx1 + batch_size
    print(f'Строки ({idx1}-{idx2-1}) ...{(14 - len(str(idx1)) - len(str(idx2-1))) * " "}', end='')
    trn_df[idx1: idx2] = trn_df[idx1: idx2].apply(fill_columns, axis=1)
    timer_stop(start, 'Готово!')

timer_stop(start0, '\nИтого: ')

Расчёт фичей в обучающей выборке

Строк в выборке: 306540
Размер батча:    50000
Всего батчей:    7

Обработка батча:
001:  Строки (0-49999) ...        Готово! (50s)
002:  Строки (50000-99999) ...    Готово! (47s)
003:  Строки (100000-149999) ...  Готово! (44s)
004:  Строки (150000-199999) ...  Готово! (46s)
005:  Строки (200000-249999) ...  Готово! (43s)
006:  Строки (250000-299999) ...  Готово! (44s)
007:  Строки (300000-349999) ...  Готово! (6s)

Итого:  (4m 41s)


In [31]:
# расчёт фичей в тестовой выборке
print('Расчёт фичей в тестовой выборке\n')
nrows = tst_df.shape[0]
start = timer_start(f'Обработка строк ({0}-{nrows-1}) ...  ')

# заполнение выборки
tst_df = tst_df.apply(fill_columns, axis=1)
timer_stop(start, 'Готово!')

Расчёт фичей в тестовой выборке

Обработка строк (0-18083) ...  Готово! (14s)


In [32]:
# out
show_dfs([trn_df, tst_df], ['train_df', 'test_df'], 1)

train_df: (306540, 1453)


,target,variantid1,variantid2,attr1,cat3_grouped,attr2,Active to Precharge Delay (tRAS),Bluetooth,CAS Latency (CL),Cтандарты HDR,FM-радио,"MTBF, кликов",OEM-номер,RAID,RAS to CAS Delay (tRCD),Row Precharge Delay (tRP),Smart TV,True Wireless,Web-камера,Автомат,Автоматика телескопа,Автоматическая двусторонняя печать,Автоматическая запись,Автоподача документов в сканер,Автослежение,...,Число трубок в комплекте,Число универсальных процессоров,"Число управляемых устройств, шт.",Число ядер процессора,Чувствительность к нажатию,"Чувствительность, дБ","Ширина загрузочного отверстия, мм","Ширина полотна, см","Ширина рабочей поверхности, см","Ширина ремешка, мм","Ширина товара в упаковке, мм",Ширина шины памяти,"Ширина, мм","Ширина, см",Шторка для веб-камеры,Шумоподавление,Шумоподавление микрофона,"Эквивалентная мощность лампы накаливания, Вт",Экран,Электробезопасность,Элементы питания,Энергопотребление,Энергоэффективность,"Эффективное количество пикселей, Мпикс","Яркость, кд/м2"
0,0.0,51197862,51198054,"{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопасность"":[""Заземление""],""Количество розеток, шт"":[""5""],...","Сетевые фильтры, разветвители и удлинители","{""Электробезопасность"":[""Заземление""],""Длина кабеля питания, м"":[""1.5""],""Входное напряжение, В"":[""250""],""Вид кабеля"":[""ПВС""],""Тип"":[""Удлинитель бытовой""],""Сечение жилы, кв.мм"":[""1""],""Количество ро...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



test_df: (18084, 1452)


,variantid1,variantid2,attr1,cat3_grouped,attr2,Active to Precharge Delay (tRAS),Bluetooth,CAS Latency (CL),Cтандарты HDR,FM-радио,"MTBF, кликов",OEM-номер,RAID,RAS to CAS Delay (tRCD),Row Precharge Delay (tRP),Smart TV,True Wireless,Web-камера,Автомат,Автоматика телескопа,Автоматическая двусторонняя печать,Автоматическая запись,Автоподача документов в сканер,Автослежение,Автофокус,...,Число трубок в комплекте,Число универсальных процессоров,"Число управляемых устройств, шт.",Число ядер процессора,Чувствительность к нажатию,"Чувствительность, дБ","Ширина загрузочного отверстия, мм","Ширина полотна, см","Ширина рабочей поверхности, см","Ширина ремешка, мм","Ширина товара в упаковке, мм",Ширина шины памяти,"Ширина, мм","Ширина, см",Шторка для веб-камеры,Шумоподавление,Шумоподавление микрофона,"Эквивалентная мощность лампы накаливания, Вт",Экран,Электробезопасность,Элементы питания,Энергопотребление,Энергоэффективность,"Эффективное количество пикселей, Мпикс","Яркость, кд/м2"
0,52076340,290590137,"{""Напряжение, В"":[""1.5""],""Бренд"":[""Perfeo""],""Тип"":[""Батарейка""],""Форм-фактор батареи"":[""AAA""],""Химический тип"":[""Щелочной""],""Страна-изготовитель"":[""Китай""],""Гарантийный срок"":[""31 день (с учетом с...",Батарейки и аккумуляторы,"{""Форм-фактор батареи"":[""AAA""],""Химический тип"":[""Щелочной""],""Назначение"":[""Универсальный""],""Комплектация"":[""Батарейка, фирменная упаковка""],""Количество в упаковке, шт"":[""4""],""Страна-изготовитель""...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br><br><br>
# Сохранение файлов с фичами, готовых для работы модели

In [33]:
%%time
# сохранение файлов с фичами
trn_df.to_parquet(files_path + trn_features_file)
tst_df.to_parquet(files_path + tst_features_file)

CPU times: user 23.9 s, sys: 1.86 s, total: 25.8 s
Wall time: 27.3 s


In [34]:
timer_stop(start_nb, 'Готово! Общее время выполнения ')

Готово! Общее время выполнения  (6m 27s)
